In [ ]:
!pip install -q koreanize-matplotlib

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Naver API 인증 정보
client_id = "Rdxqa5dG01AumJcUJFg8"
client_secret = "7RfDqVFoEn"

url = "https://openapi.naver.com/v1/datalab/search"

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret,
    "Content-Type": "application/json"
}

def get_keyword_trend(keywords, start_date="2024-01-01", end_date="2024-12-01", time_unit="month"):
    """
    keywords: ['티셔츠', '패딩'] 처럼 개별 키워드 리스트
    반환: date(YYYY-MM 형태), keyword, ratio 컬럼을 갖는 DataFrame
    """
    keyword_groups = [{"groupName": kw, "keywords": [kw]} for kw in keywords]
    body = {
        "startDate": start_date,
        "endDate": end_date,
        "timeUnit": time_unit,
        "keywordGroups": keyword_groups,
        "gender": "m"
    }

    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 200:
        data = response.json()
        results = []
        for item in data["results"]:
            group_name = item["title"]
            for point in item["data"]:
                results.append({
                    "date": point["period"],
                    "keyword": group_name,
                    "ratio": point["ratio"]
                })
        return pd.DataFrame(results)
    else:
        print(f"API 요청 실패: {response.status_code}, {response.text}")
        return pd.DataFrame()

# ---------------------------
# other_keywords만 사용한 데이터 수집
# ---------------------------
other_keywords = [
    "티셔츠", "맨투맨", "패딩", "양말", "스카프",
    "청바지", "가디건", "스니커즈", "자켓", "수트"
]

all_list = []
for kw in other_keywords:
    df_temp = get_keyword_trend(
        keywords=[kw],  # 개별 키워드
        start_date="2024-01-01",
        end_date="2024-12-01",
        time_unit="month"
    )
    if df_temp.empty:
        continue

    df_temp["month"] = df_temp["date"]
    all_list.append(df_temp)

# ---------------------------
# 데이터 합치기
# ---------------------------
if not all_list:
    print("데이터가 전혀 수집되지 않았습니다. (모두 실패)")
    exit()

combined_df = pd.concat(all_list, ignore_index=True)

# ---------------------------
# 월별 ratio 표 만들기
# ---------------------------
pivot_df = combined_df.pivot_table(
    index="month",
    columns="keyword",
    values="ratio",
    aggfunc="mean"
)

pivot_df.sort_index(inplace=True)

print("=== 월별 ratio 표 (other_keywords만 사용) ===")
display(pivot_df)

# CSV 저장
pivot_df.reset_index().to_csv("other_keywords_ratio.csv", index=False, encoding="utf-8-sig")
print("\n월별 키워드 ratio를 'other_keywords_ratio.csv'로 저장했습니다.")

# ---------------------------
# 상관분석 수행
# ---------------------------
corr_matrix = pivot_df.corr(method="pearson")

print("=== 키워드 전체 상관행렬 (other_keywords만) ===")
display(corr_matrix)

# CSV로 저장
corr_matrix.to_csv("other_keywords_correlation_matrix.csv", encoding="utf-8-sig")
print("\n키워드 전체 상관행렬을 'other_keywords_correlation_matrix.csv'로 저장했습니다.")

# ---------------------------
# 상관계수 히트맵 시각화
# ---------------------------
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    linewidths=0.5,
    cbar=True
)
plt.title("Other Keywords Correlation Matrix")
plt.tight_layout()
plt.savefig("other_keywords_correlation_heatmap.png", dpi=300)
plt.show()

print("\n상관계수 히트맵이 'other_keywords_correlation_heatmap.png'로 저장되었습니다.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.1 MB/s eta 0:00:00


In [ ]:
# 내가 원하는건 네이버 데이터랩에 무신사티셔츠 무신사양말 무신사패딩 이런식으로 base_keyword + other_keywords를 더해서 총 10개를 만드는거야. 이 코드에서는 그러고 있는지 봐주고, 그렇지 않다면 코드를 수정하고, 전체 코드를 수정해서 다시 반환해줘.

!pip install -q koreanize-matplotlib

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

client_id = "Rdxqa5dG01AumJcUJFg8"
client_secret = "7RfDqVFoEn"

url = "https://openapi.naver.com/v1/datalab/search"

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret,
    "Content-Type": "application/json"
}

def get_keyword_trend(keywords, start_date="2024-01-01", end_date="2024-12-01", time_unit="month"):
    """
    keywords: ['무신사티셔츠', '무신사패딩'] 처럼 조합된 키워드 리스트
    반환: date(YYYY-MM 형태), keyword, ratio 컬럼을 갖는 DataFrame
    """
    keyword_groups = []
    for kw in keywords:
        keyword_groups.append({
            "groupName": kw,
            "keywords": [kw]
        })

    body = {
        "startDate": start_date,
        "endDate": end_date,
        "timeUnit": time_unit,  # 'month'
        "keywordGroups": keyword_groups,
        "gender": "m"
    }

    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 200:
        data = response.json()
        results = []
        for item in data["results"]:
            group_name = item["title"]
            for point in item["data"]:
                results.append({
                    "date": point["period"],
                    "keyword": group_name,
                    "ratio": point["ratio"]
                })
        return pd.DataFrame(results)
    else:
        print("API 요청 실패:", response.status_code, response.text)
        return pd.DataFrame()  # 빈 DF 반환

# ---------------------------
# 1) 무신사 + 10개 키워드 각각 월별 데이터 수집
# ---------------------------
base_keyword = "무신사"
other_keywords = [
    "티셔츠", "맨투맨", "패딩", "양말", "스카프",
    "청바지", "가디건", "스니커즈", "자켓", "수트"
]

# 무신사 + 다른 키워드 조합 생성
combined_keywords = [base_keyword + kw for kw in other_keywords]

all_list = []  # 전체 결과를 모을 리스트

for kw in combined_keywords:
    df_temp = get_keyword_trend(
        keywords=[kw],  # 하나의 조합 키워드만 전달
        start_date="2024-01-01",
        end_date="2024-12-01",
        time_unit="month"
    )
    if df_temp.empty:
        continue

    df_temp["month"] = df_temp["date"]
    all_list.append(df_temp)

# ---------------------------
# 2) 전체 데이터 합치기
# ---------------------------
if not all_list:
    print("데이터가 전혀 수집되지 않았습니다. (모두 실패)")
    exit()

combined_df = pd.concat(all_list, ignore_index=True)

# ---------------------------
# 3) 월별 ratio 표 만들기
# ---------------------------
pivot_df = combined_df.pivot_table(
    index="month",
    columns="keyword",
    values="ratio",
    aggfunc="mean"
)

pivot_df.sort_index(inplace=True)

print("=== 월별 ratio 표 (무신사 + 조합 키워드) ===")
display(pivot_df)

# CSV 저장
pivot_df.reset_index().to_csv("month_combined_keyword_ratio.csv", index=False, encoding="utf-8-sig")
print("\n월별 키워드 ratio를 'month_combined_keyword_ratio.csv'로 저장했습니다.")

# ---------------------------
# 4) 상관분석 (12개월 시계열 기반)
# ---------------------------
corr_matrix = pivot_df.corr(method="pearson")

print("=== 키워드 전체 상관행렬 ===")
display(corr_matrix)

# CSV로 저장
corr_matrix.to_csv("combined_keyword_correlation_matrix.csv", encoding="utf-8-sig")
print("\n키워드 전체 상관행렬을 'combined_keyword_correlation_matrix.csv'로 저장했습니다.")

# ---------------------------
# 5) 상관계수 히트맵 시각화
# ---------------------------
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    linewidths=0.5,
    cbar=True
)
plt.title("Combined Keyword Correlation Matrix")
plt.tight_layout()
plt.savefig("combined_keyword_correlation_heatmap.png", dpi=300)
plt.show()

print("\n상관계수 히트맵이 'combined_keyword_correlation_heatmap.png'로 저장되었습니다.")
